# 필수 라이브러리 호출

In [ ]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import random

## contentGenre 페이지 내에서 웹툰 random 추출 후 2015년 이후 작품 선별

In [ ]:
contentGenreNameList = ['daily', 'comic', 'fantasy', 'action', 'drama', 'pure', 'sensibility', 'thrill', 'historical', 'sports']
contentGenrePage = np.array([41,30,47,18,39,29,18,13,5,3]) #각 contentGenre별 마지막 페이지수
sum(contentGenrePages*9) #각 페이지별로 9개씩 뽑을 예정!

In [ ]:
webtoon = pd.DataFrame(columns= ['title','titleId','isPublic','totalStar','contentGenre','typeGenre','startDay',
                                 't1','star1','starPar1','views1',
                                 't2','star2','starPar2','views2', 'day2'
                                 't3','star3','starPar3','views3', 'day3']) #크롤링 데이터 저장할 dataframe 생성

In [ ]:
for j,contentGenre in enumerate(contentGenreNameList):
    for page in range(1,contentGenrePage[j]+1):        
        url = "https://comic.naver.com/genre/bestChallenge?m="+contentGenre+"&order=ViewCount&page="+str(page)
        req = Request(url)
        response = urlopen(req)
        page = response.read()
        bsobj = BeautifulSoup(page, "html.parser")
        time.sleep(random.uniform(1,3))
        
        chall_img = bsobj.find("div",{"id":"content"}).findAll("div",{"class":"fl"})
        chall_info = bsobj.find("div",{"id":"content"}).findAll("div",{"class":"challengeInfo"})
        
        random.seed(page)
        randomInt = random.sample(range(len(chall_info)),min(8,len(chall_info))) #마지막 페이지에 웹툰이 8개 미만일시 모두 크롤링.
        ## 제목, 전체별점, 장르, 정식연재인가 크롤링
        for i in randomInt:
            titleId = re.findall('\d+', chall_img[i].find("a").attrs['href'])[0]
            ## 중복되는 웹툰 스킵!(장르만 추가)
            isdup = False
            for idx in range(len(webtoon)):
                if webtoon.loc[idx, 'titleId'] == titleId:
                    webtoon.loc[idx,'contentGenre'] += ', ' + contentGenre
                    isdup = True
                    break
            if isdup: continue
                
            webtoon.loc[len(webtoon), 'title'] = chall_info[i].find("a").get_text()
            webtoon.loc[len(webtoon)-1, 'totalStar'] = chall_info[i].find("strong").get_text()
            webtoon.loc[len(webtoon)-1, 'contentGenre'] = contentGenre
            if (str(chall_img[i].find("span",{"class":"mark_serial"})) == "None"): #비정식연재인 경우
                webtoon.loc[len(webtoon)-1, 'isPublic'] = 0
            else : webtoon.loc[len(webtoon)-1, 'isPublic'] = 1 #정식연재인 경우
            webtoon.loc[len(webtoon)-1, 'titleId'] = titleId
    print(contentGenre,'완료')

In [ ]:
webtoon.head(6)

In [ ]:
webtoon.tail(6)

### 연재 시작일 크롤링

In [ ]:
wd = webdriver.Chrome('./chromedriver.exe')

for i in range(len(webtoon)):
    url = "https://comic.naver.com/bestChallenge/list?titleId=" + webtoon.loc[i,'titleId']
    wd.get(url)
    time.sleep(random.uniform(1,3))

    ## 1화
    try: # 게시물 신고에 의한 alert 웹툰 제외하고 크롤링
        wd.find_elements(By.CSS_SELECTOR, "#content > div.comicinfo > div.detail > ul > li:nth-child(2) > a > span")[0].click()
        alert = Alert(wd); alert.accept()
        wd.back(); wd.back()
        continue
    except:
        webtoon.loc[i,'startDay'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)")[0].text
    time.sleep(random.uniform(1,4))
    if i%50==0:
        print(i+1,'개 완료')

In [ ]:
webtoon['startDay'].isnull().sum() #다 잘 뽑힘.

In [ ]:
webtoon.tail(1)

In [ ]:
# 데이터 타입 수정
webtoon['startDay'] = pd.to_datetime(webtoon['startDay'])
webtoon = webtoon.astype({'isPublic':bool,'totalStar':float})

In [ ]:
# 2015년 이전 작품 삭제하여 최종 크롤링할 작품만 남김.
webtoon = webtoon.loc[webtoon['startDay'].dt.year>2015].copy()

### 비정식 연재 크롤링

In [ ]:
# 추가 크롤링을 위한 열추가
webtoon = webtoon.reindex(columns = webtoon.columns.tolist() + ["t4","star4","starPar4","views4",'day4',
                                                                "t5","star5","starPar5","views5",'day5',
                                                                "t6","star6","starPar6","views6",'day6',"heart"])

In [ ]:
# webtoon에서 비정식 연재 데이터만 추출
webtoon['isPublic'] = webtoon['isPublic'].astype('int') #정식연재여부를 (object)에서 string type으로 바꿈.

publicIndex = webtoon[webtoon['isPublic'] == 1].index
notPublicIndex = webtoon[webtoon['isPublic'] == 0].index

In [ ]:
#정식/비정식 웹툰 titleId
publicTitleId = webtoon.loc[publicIndex,'titleId']
notPublicTitleId = webtoon.loc[notPublicIndex,'titleId']

In [ ]:
# 댓글 크롤링할 데이터프레임
comment= pd.DataFrame(columns= ['titleId','order','comment1','like','hate'])
comment['titleId'] = webtoon['titleId'].copy() #webtoon과 동일한 index 사용

In [ ]:
wd = webdriver.Chrome('./chromedriver.exe')
for i,titleId in zip(publicIndex, publicTitleId):
    url = "https://comic.naver.com/bestChallenge/list?titleId="+ str(titleId)
    wd.get(url)
    time.sleep(4)
    
    webtoon.loc[i,'heart'] = wd.find_element(By.CLASS_NAME, "u_cnt").text
    webtoon.loc[i,'typeGenre'] = wd.find_element(By.CSS_SELECTOR,'#content > div.snb > ul > li.on').text
    
    # 뒤에서 1화
    if len(wd.find_elements(By.CSS_SELECTOR,'#content > table > tbody > tr.band_banner.v2 > td > a > em'))>0:
        #시리즈온 작품때문에 뒤에서 1화의 클릭 위치가 다른 경우 처리.
        wd.find_element(By.CSS_SELECTOR, "#content > table > tbody > tr:nth-child(2) > td.title > a").click()
    else:
        wd.find_element(By.CSS_SELECTOR, "#content > table > tbody > tr:nth-child(1) > td.title > a").click()
    time.sleep(random.uniform(1,4))    
   
    webtoon.loc[i,'t6'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star6'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar6'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views6'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day6'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)): #댓글수가 5개 미만인 경우 고려
        comment.loc[5*(6*i)+j,'titleId'] = titleId
        comment.loc[5*(6*i)+j,'order'] = -1
        comment.loc[5*(6*i)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > div > span > a").click() #이전화 이동
    time.sleep(random.uniform(1,4))
    
    # 뒤에서 2화
    webtoon.loc[i,'t5'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star5'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar5'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views5'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day5'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+1)+j,'titleId'] = titleId
        comment.loc[5*(6*i+1)+j,'order'] = -2
        comment.loc[5*(6*i+1)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+1)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+1)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > div > span > a").click() #이전화 이동
    time.sleep(random.uniform(1,4))
    
    # 뒤에서 3화
    webtoon.loc[i,'t4'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star4'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar4'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views4'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day4'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+2)+j,'titleId'] = titleId
        comment.loc[5*(6*i+2)+j,'order'] = -3
        comment.loc[5*(6*i+2)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+2)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+2)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    
    # 첫회 : 댓글
    wd.find_element(By.CSS_SELECTOR,"#comicNavigation > div.btn_group > div > a.first").click() #첫화 이동
    time.sleep(random.uniform(1,4))
    webtoon.loc[i,'t1'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star1'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar1'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views1'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+3)+j,'titleId'] = titleId
        comment.loc[5*(6*i+3)+j,'order'] = 1
        comment.loc[5*(6*i+3)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+3)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+3)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR,"#sectionContWide > div.tit_area > div.view > div > span.next > a").click() #다음화 이동
    time.sleep(random.uniform(1,4))
    
    # 2화 : 등록일 + 댓글
    webtoon.loc[i,'day2'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    webtoon.loc[i,'t2'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star2'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar2'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views2'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+4)+j,'titleId'] = titleId
        comment.loc[5*(6*i+4)+j,'order'] = 2
        comment.loc[5*(6*i+4)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+4)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+4)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR,"#sectionContWide > div.tit_area > div.view > div > span.next > a").click() #다음화 이동
    time.sleep(random.uniform(1,4))
    
    #3화 : 동록일 + 댓글
    webtoon.loc[i,'day3'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    webtoon.loc[i,'t3'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star3'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar3'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views3'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+5)+j,'titleId'] = titleId
        comment.loc[5*(6*i+5)+j,'order'] = 3
        comment.loc[5*(6*i+5)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+5)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+5)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()

### 정식연재

In [ ]:
lastno = [81,21,35,22,10,56,3,9,-1,-1,21,25,34,11,14,17,11,-1,-1,21,6,11,18,13,7,7,25,11,23,-1,20,8,8,6,7,-1,
          34,10,6,28,-1,5,75,12,-1,14,6,11,20,23,-1,4,8,16,15,6,3,-1,5,7,-1,25,15,4,18,6,17,13,19,21,6,13,-1,
          9,18,1,8,13,13,5,-1,17,14] #공지를 제외한 마지막 화 url 끝부분의 숫자
                                     #공지밖에 없는 웹툰들은 불러오지 않음. -> -1로 표현
lastno = pd.DataFrame(np.array(lastno).astype(int), index = publicIndex)
lastno.columns = ['lastno']

In [ ]:
wd = webdriver.Chrome('./chromedriver.exe')
for i,titleId in zip(publicIndex, publicTitleId):
    #lastno가 np.nan이면 continue 후 나중에 webtoon 에서 na존재하는 행 삭제
    if lastno.loc[i,'lastno'] == -1:
        continue
    url = "https://comic.naver.com/bestChallenge/list?titleId=" + str(titleId)
    wd.get(url)
    time.sleep(3)

    webtoon.loc[i,'heart'] = wd.find_element(By.CLASS_NAME, "u_cnt").text
    webtoon.loc[i,'typeGenre'] = wd.find_element(By.CSS_SELECTOR,'#content > div.snb > ul > li.on').text

    # 뒤에서 1화
    url = "https://comic.naver.com/bestChallenge/detail?titleId="+ str(titleId) + "&no=" + str(lastno.loc[i,'lastno'])
    wd.get(url)
    time.sleep(random.uniform(1,4))
    webtoon.loc[i,'t6'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star6'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar6'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views6'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day6'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)): #댓글수가 5개 미만인 경우 고려
        comment.loc[5*(6*i)+j,'titleId'] = titleId
        comment.loc[5*(6*i)+j,'order'] = -1
        comment.loc[5*(6*i)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > div > span > a").click() #이전화 이동
    time.sleep(random.uniform(1,4))
    
    # 뒤에서 2화
    webtoon.loc[i,'t5'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star5'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar5'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views5'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day5'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+1)+j,'titleId'] = titleId
        comment.loc[5*(6*i+1)+j,'order'] = -2
        comment.loc[5*(6*i+1)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+1)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+1)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > div > span > a").click() #이전화 이동
    time.sleep(random.uniform(1,4))
    
    # 뒤에서 3화
    webtoon.loc[i,'t4'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star4'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar4'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views4'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    webtoon.loc[i,'day4'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    wd.switch_to.frame('commentIframe') # 댓글 크롤링
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+2)+j,'titleId'] = titleId
        comment.loc[5*(6*i+2)+j,'order'] = -3
        comment.loc[5*(6*i+2)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+2)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+2)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    
    # 첫회 : 댓글
    wd.find_element(By.CSS_SELECTOR,"#comicNavigation > div.btn_group > div > a.first").click() #첫화 이동
    time.sleep(random.uniform(1,4))
    webtoon.loc[i,'t1'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star1'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar1'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views1'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+3)+j,'titleId'] = titleId
        comment.loc[5*(6*i+3)+j,'order'] = 1
        comment.loc[5*(6*i+3)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+3)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+3)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR,"#sectionContWide > div.tit_area > div.view > div > span.next > a").click() #다음화 이동
    time.sleep(random.uniform(1,4))
    
    # 2화 : 등록일 + 댓글
    webtoon.loc[i,'day2'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    webtoon.loc[i,'t2'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star2'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar2'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views2'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+4)+j,'titleId'] = titleId
        comment.loc[5*(6*i+4)+j,'order'] = 2
        comment.loc[5*(6*i+4)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+4)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+4)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()
    wd.find_element(By.CSS_SELECTOR,"#sectionContWide > div.tit_area > div.view > div > span.next > a").click() #다음화 이동
    time.sleep(random.uniform(1,4))
    
    #3화 : 동록일 + 댓글
    webtoon.loc[i,'day3'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(2)')[0].text
    webtoon.loc[i,'t3'] = wd.find_elements(By.CSS_SELECTOR, "#sectionContWide > div.tit_area > div.view > h3")[0].text
    webtoon.loc[i,'star3'] = wd.find_elements(By.CSS_SELECTOR, '#topPointTotalNumber')[0].text
    webtoon.loc[i,'starPar3'] = wd.find_elements(By.CSS_SELECTOR, '#topTotalStarPoint > span.pointTotalPerson > em')[0].text
    webtoon.loc[i,'views3'] = wd.find_elements(By.CSS_SELECTOR, '#sectionContWide > div.tit_area > div.vote_lst > dl.rt > dd:nth-child(4)')[0].text
    wd.switch_to.frame('commentIframe')
    for j in range(min(len(wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')),5)):
        comment.loc[5*(6*i+5)+j,'titleId'] = titleId
        comment.loc[5*(6*i+5)+j,'order'] = 3
        comment.loc[5*(6*i+5)+j,'comment'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_contents')[j].text
        comment.loc[5*(6*i+5)+j,'like'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_recomm')[j].text
        comment.loc[5*(6*i+5)+j,'hate'] = wd.find_elements(By.CLASS_NAME, 'u_cbox_cnt_unrecomm')[j].text
    wd.switch_to.default_content()

In [ ]:
webtoon.shape

In [ ]:
webtoon.loc[webtoon['isPublic']==1].shape

### Data 저장

In [ ]:
## 웹툰 데이터 csv 파일로 저장
webtoon.to_csv('webtoon.csv', index= False)
comment.to_csv('comment.csv', index= False)

In [ ]:
# 저장 확인
import pandas as pd

webtoon = pd.read_csv('webtoon.csv')
comment = pd.read_csv('comment.csv')